In [ ]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.python.framework import ops
import random

In [82]:
gHidden1OutputSize = 20
pHidden1OutputSize = 50
sHidden1OutputSize = 50
hidden2OutputSize = 1
seed = 0
batchSize = 10
trainingRatio = 

In [87]:
#Load the features into arrays
inputFile = 'features.pckl'
with open(inputFile, 'rb') as fid:
    features = pickle.load(fid)
random.shuffle(features)
numSamples = len(features)


In [ ]:
def splitAndBatchFeatures(batchSize, features):
    random.shuffle(features)
    globalFeatures = np.array([feature['globalFeatures'] for feature in features], dtype='float32')
    pieceFeatures  = np.array([feature['pieceFeatures'] for feature in features], dtype='float32')
    squareFeatures = np.array([feature['squareFeatures'] for feature in features], dtype='float32')
    ranks = np.array([feature['moveRankings'] for feature in features], dtype='float32')
    labels = np.array([feature['labels'] for feature in features], dtype='float32')

    globalLength= globalFeatures.shape[1]
    pieceLength = pieceFeatures.shape[1]
    squareLength = squareFeatures.shape[1]
    #reshape the arrays into batches
    numSamples = globalFeatures.shape[0]
    goalNumSamples = numSamples - numSamples%batchSize
    #trim samples to fit evenly into batches
    globalFeaturesTrim = globalFeatures[:goalNumSamples, :]
    pieceFeaturesTrim = pieceFeatures[:goalNumSamples, :]
    squareFeaturesTrim = squareFeatures[:goalNumSamples, :]
    ranksTrim = ranks[:goalNumSamples, :]
    labelsTrim = labels[:goalNumSamples, :]

    #reshape into batches
    globalBatchFeed = globalFeatruresTrim.reshape(-1, batchSize, globalLength)
    piecelBatchFeed = pieceFeatruresTrim.reshape(-1, batchSize, pieceLength)
    squareBatchFeed = squareFeatruresTrim.reshape(-1, batchSize, squarelLength)
    rankBatchFeed = ranksTrim.reshape(-1, batchSize, 1)
    labelBatchFeed = labelsTrim.reshape(-1, batchSize, 1)
    
    return globalBatchFeed, pieceBatchFeed, squareBatchFeed, rankBatchFeed, labelBatchFeed

#Test
splitAndBatchFeatures(batchSize, trainingFeatures)

In [89]:
tf.reset_default_graph()

'''globalTensor = tf.convert_to_tensor(globalFeatures, dtype=tf.float32)
pieceTensor = tf.convert_to_tensor(pieceFeatures, dtype=tf.float32)
squareTensor = tf.convert_to_tensor(squareFeatures, dtype=tf.float32)
rankTensor = tf.convert_to_tensor(ranks, dtype=tf.float32)
labelTensor = tf.convert_to_tensor(labels, dtype=tf.float32)

train_input_queue = tf.train.slice_input_producer(
                                    [globalFeatures, 
                                     pieceFeatures, 
                                     squareFeatures,
                                     ranks,
                                     labels],
                                    shuffle=False)

globalBatch, pieceBatch, squareBatch, rankBatch, labelBatch = tf.train.batch(
                                    train_input_queue,
                                    batch_size=batchSize
                                    #,num_threads=1
                                    )
'''
#global variables
with tf.variable_scope('GlobalVariables'):
    globalBatch = tf.placeholder(tf.float32, shape=(batchSize, globalLength), name='GlobalBatch')#1 bit (everything is really just floats)
    #global hidden layer
    gHidden1 = tf.layers.dense(globalBatch, gHidden1OutputSize, name='GlobalHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
    #testWeights = tf.get_variable("kernel")
#piece-centric variables
with tf.variable_scope('Piece-CentricVariables'):
    pieceBatch = tf.placeholder(tf.float32, shape=(batchSize, pieceLength), name='PieceBatch')

    #piece-centric hidden layer
    pHidden1 = tf.layers.dense(pieceBatch, pHidden1OutputSize, name='Piece-CentricHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

#square-centric variables
with tf.variable_scope('Sqaure-CentricVariables'):
    squareBatch = tf.placeholder(tf.float32, shape=(batchSize, squareLength), name='SquareBatch')
    rankBatch = tf.placeholder(tf.float32, shape=(batchSize, 1), name='RankBatch')
    squareRankBatch = tf.concat([squareBatch, rankBatch], 1)
    #square-centric hidden layer
    sHidden1 = tf.layers.dense(squareRankBatch, sHidden1OutputSize, name='Square-CentricHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

input2 = tf.concat([gHidden1, pHidden1, sHidden1], 1)
logits = tf.layers.dense(input2, hidden2OutputSize, name='HiddenLayer2', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
probability = tf.nn.sigmoid(logits)
labelBatch = tf.placeholder(tf.float32, shape=(batchSize, 1), name='LabelBatch')
loss = tf.losses.sigmoid_cross_entropy(labelBatch, logits) #groundTruth must be of shape [batchSize, 1]
optimizer = tf.train.AdadeltaOptimizer()
trainer = optimizer.minimize(loss)

In [90]:
globalFeatures.shape


(89, 19)

In [96]:
batchesPerEpoch = float(globalFeatures.shape[0])/batchSize
numEpochs = 1
saveDir = '/home/rvansoelen/rbmcData/savedModels/test/'
saveFrequency = 1000
saver = tf.train.Saver()
lastSavePath = ''
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #print sess.run( weights = tf.get_variable("GlobalVariable/kernel"))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    lossRecord = []
    for epoch in range(1):
        globalBatchFeed, pieceBatchFeed, squareBatchFeed, rankBatchFeed, labelBatchFeed = splitAndBatchFeatures(batchSize, trainingFeatures)
        for i in range(globalBatchFeed.shape[0]):
            thisLoss = sess.run((loss, trainer), 
                feed_dict={globalBatch=globalBatchFeed[i], 
                          pieceBatch=pieceBatchFeed[i], 
                          squareBatch=squareBatchFeed[i], 
                          rankBatch=rankBatchFeed[i], 
                          labelBatch=labelBatchFeed[i]
                          })[0]
            lossRecord.append(thisLoss)
            if i % saveFrequency == 0:
                path = saveDir+'model_'+str(i)+'.ckpt'
                lastSavePath = saver.save(sess, path)
    coord.request_stop()
    coord.join(threads)
    coord.request_stop()
    coord.join(threads)
    '''tf.saved_model.simple_save(sess,
            saveDir,
            inputs={'globalFeatures': train_input_queue[0],
                    'pieceFeatures': train_input_queue[1], 
                    'squareFeatures': train_input_queue[2],
                    'ranks' : train_input_queue[3], 
                    'labels' : train_input_queue[4]},
            outputs={"probability": probability})'''


In [97]:
lossRecord

[1.3263891, 1.5419257]

In [99]:
#evaluate training data
#Does something have to be disabled to stop learning??
fp = 0
fn = 0
tp = 0
tn = 0
lossRecord = []
predictionRecord = []
with tf.Session() as sess:
    # Restore variables from disk.
    i=0
    #path = saveDir+'model_'+str(i)+'.ckpt'
    saver.restore(sess, lastSavePath)
    globalBatchFeed, pieceBatchFeed, squareBatchFeed, rankBatchFeed, labelBatchFeed = splitAndBatchFeatures(batchSize, trainingFeatures)
    for i in range(globalBatchFeed.shape[0]):
        thisProbability, thisLoss = sess.run((probability, loss), 
            feed_dict={globalBatch=globalBatchFeed[i], 
                      pieceBatch=pieceBatchFeed[i], 
                      squareBatch=squareBatchFeed[i], 
                      rankBatch=rankBatchFeed[i], 
                      labelBatch=labelBatchFeed[i]
                      })
        lossRecord.append(thisLoss)
        predictionRecord.extend([(probability[index], label[index]) for range(batchSize)])

accuracy = []
recall = []
precision = []
f1 = []
for threshold in linspace(start, stop, num=50):
    fp = 0
    fn = 0
    tp = 0
    tn = 0
    for prob, label in predictionRecord:
        fp += np.sum(np.logical_and(prob <  0.5, label >= 0.5))
        fn += np.sum(np.logical_and(prob >= 0.5, label <  0.5))
        tp += np.sum(np.logical_and(prob >= 0.5, label >= 0.5))
        tn += np.sum(np.logical_and(prob <  0.5, label <  0.5))
    accuracy.append(float(tp+tn)/(fp+fn+tp+tn))
    recall.append(float(tp)/(tp+fn))
    precision.append(float(tp)/tp+fp)
    f1.append(2*recall[-1]*precision[-1]/(recall[-1]+precision[-1]))


INFO:tensorflow:Restoring parameters from /home/rvansoelen/rbmcData/savedModels/test/model_0.ckpt


In [ ]:
import matplotlib.pyplot as plt
plt.plot(accuracy)
plt.title('Accuracy')
plt.show()
plt.plot(f1)
plt.title('F1 Score')
plt.show()
plt.plot(precision, recall)
plt.title('ROC Curve')
plt.ylabel('Recall')
plt.xlabel('Precision')
plt.show()